In [1]:
import json, boto3, logging
from botocore.exceptions import ClientError

In [3]:
with open("../.KEYS/API_KEYS.json", "r") as f:
    key = json.load(f)

# S3 버킷 정보 get
with open("../.KEYS/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)

with open("../.KEYS/GEMINI_API_KEY.json", "r") as f:
    gemini_api_key = json.load(f)

In [4]:
dynamo_table_name = bucket_info['restore_table_name']
dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id=key['aws_access_key_id'],
        aws_secret_access_key=key['aws_secret_key'],
        region_name=key['region']
    )
table = dynamodb.Table(dynamo_table_name)

scan_kwargs = {}
data = []

In [ ]:
# Pagination을 고려하여 모든 데이터를 가져오기
while True:
    response = table.scan(**scan_kwargs)
    data.extend(response.get('Items', []))
    scan_kwargs['ExclusiveStartKey'] = response.get('LastEvaluatedKey', None)
    if not scan_kwargs['ExclusiveStartKey']:
        break